<a href="https://colab.research.google.com/github/takumiyamaoka/AzureIoTDevices/blob/master/jira_calculate_kpi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# @title Google認証後、recruitment_channel から最新の採用チャネル情報を取得
# @markdown ※チャネルに変更があった場合は随時更新すること！下の出力結果を確認してください
!pip install --upgrade gspread
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

print(gspread.__version__)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

ss_id = '1X0zXliPrfmwpY_0G3wHn7BQxOeclTrVodg9Jvgofgx0'
workbook = gc.open_by_key(ss_id)
worksheet = workbook.worksheet('recruitment_channel')

df_category = pd.DataFrame(worksheet.get_all_records())
df_category

6.1.4


,採用チャネル,カテゴリ
0,公式HP,その他
1,リファラル,その他
2,Wantedly,スカウト
3,LinkedIn,スカウト
4,ビズリーチ,スカウト
5,doda,エージェント
6,リクナビ,エージェント
7,エッジテクノロジー,その他
8,bloom,エージェント
9,Facebook,スカウト


In [2]:
# @title Recruitment JIRAから必要な情報を取得
# @markdown base_dateには集計の「基準日」を入力する
# @markdown - base_dateを変更して実行することで、「その日時点の採用状況」がわかる
# @markdown - Weeklyで採用定例等に使いたい場合は、定例実施日の前営業日をbase_dateにすると良い

import requests
from requests.auth import HTTPBasicAuth
import json
from datetime import datetime, timedelta

jira_url = "https://trust-com.atlassian.net/rest/api/3"
auth = HTTPBasicAuth("takumi_yamaoka@trust-partner.co.jp", "ATATT3xFfGF0oGZ6AoDpqR7QI6EbotLPW6QQr0WXI-GOTP7AwqT3c6YdF_Rj92AzpuRmUWFEIvSFykjJ8MRaVcr0H778T38JwsybzQE13V08Vzb3EMNjLBGiDpQfMcdaJaYYuu7aG4BUeeqcXGv5I1DyBM-1Q9yjXsHROhepKp_bKvrk1jNIxko=040CF590")

base_date = "2024-12-06" # @param {"type":"date"}
project_id = 'RECRT'

fields_to_be_returned = 'status,customfield_10082,customfield_10083,customfield_10084,customfield_10085,created'

headers = {
    "Accept": "application/json"
}

In [3]:
# @title Declare Functions
def get_jql(key=None, last7days_flg=None):
    if(key):
        jql_created_date = jql_during = ''
        try:
            # Convert base_date to a datetime object
            base_date_dt = datetime.strptime(base_date, '%Y-%m-%d')

            if last7days_flg:
                # Calculate the start date (7 days before the base_date) and use a 7-day range
                start_date = (base_date_dt - timedelta(days=6)).strftime('%Y-%m-%d')
                jql_created_date = f'AND created >= "{start_date}" AND created <= "{base_date}"'
                jql_during = f'DURING ("{start_date}", "{base_date}")'
            else:
                # Use a single date condition to filter created issues before base_date
                jql_created_date = f'AND created <= "{base_date}"'
                # Use BEFORE clause for transitions before the base_date
                jql_during = f'BEFORE "{base_date}"'
        except ValueError:
            raise ValueError("Invalid date format. Please provide the date in 'YYYY-MM-DD' format.")

        jql = {
            'candidates': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 {jql_created_date}',
            'applied': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND status changed TO 10037 {jql_during}',
            'screening_passed': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND status changed FROM 10037 TO 10038 {jql_during}',
            'screening_rejected': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND (status changed FROM 10037 TO 10041 {jql_during} OR status changed from 10077 to 10041 {jql_during})',
            'interview1_passed': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND status changed FROM 10038 TO 10064 {jql_during}',
            'interview1_rejected': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND (status changed FROM 10038 TO 10041 {jql_during} OR status changed from 10038 TO 10081 {jql_during})',
            'final_passed': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND status changed FROM 10064 TO 10040 {jql_during}',
            'final_rejected': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND (status changed FROM 10064 TO 10041 {jql_during} OR status changed from 10064 TO 10081 {jql_during})',
            'offer_accepted': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND status changed FROM 10039 TO 10042 {jql_during}',
            'offer_rejected': f'project = {project_id} AND "雇用形態[Dropdown]" = 正社員 AND status changed FROM 10039 TO 10065 {jql_during}'
        }
    return jql[key]

### Search for issues using JQL (GET)
### https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-issue-search/#api-rest-api-3-search-get
def get_issues(key, last7days_flg=None):
    url = f"{jira_url}/search"
    all_issues = []
    start_at = 0
    JIRA_MAX_RESULTS = 100
    candidate_info = [] # List to store extracted information

    # if base_date:
    #     jql += ' ' + get_created_date_jql(base_date, last7days_flg)

    while True:
        # JQL query with pagination
        query = {
            'jql': get_jql(key, last7days_flg),
            'startAt': start_at,
            'maxResults': JIRA_MAX_RESULTS,
            'fields': fields_to_be_returned
        }

        # Send the request to Jira
        response = requests.get(url, headers=headers, params=query, auth=auth)

        if response.status_code == 200:
            # Load the JSON response
            data = json.loads(response.text)
            # Iterate over the issues and extract the necessary fields
            for issue in data["issues"]:
                fields = issue["fields"]

                # Extracting necessary information
                candidate = {
                    "Issue ID": issue["key"],
                    "ポジション": fields.get("customfield_10082", {}).get("value", None),
                    "応募日": fields.get("customfield_10083", None),
                    "雇用形態": fields.get("customfield_10084", {}).get("value", None),
                    "採用チャネル": fields.get("customfield_10085", {}).get("value", None),
                    "ステータス": fields.get("status", {}).get("name", None),
                    # "created": fields.get("created", None
                    "last7days_flg": last7days_flg
                }
                # Add the extracted information to the list
                candidate_info.append(candidate)

            # Check if there are more issues to fetch
            if len(data['issues']) < JIRA_MAX_RESULTS:
                # We have retrieved all the issues, break the loop
                break

            # Update the startAt value to fetch the next page
            start_at += JIRA_MAX_RESULTS
        else:
            print(f"Failed to retrieve issues. Status code: {response.status_code}, Response: {response.text}")
            break

    # Create a DataFrame from the extracted candidate information
    df = pd.DataFrame(candidate_info)

    return df

def get_issues_with_flag(key=None):
    if(key):
        df_recent = get_issues(key, True)
        df_all = get_issues(key, False)
        df_union = pd.concat([df_recent,df_all])
        return df_union.drop_duplicates(subset='Issue ID', keep='first')

def format_count(total, last7days):
    if last7days > 0:
        return f"{total} (+{last7days})"
    else:
        return f"{total}"

def calculate_pass_rate(df_positive, df_negative, prefix, key):
    # Group 'df_positive' by 'key' and aggregate counts
    passed_grouped = df_positive.groupby(key).agg(
        positive_count=('Issue ID', 'count'),
        positive_count_last7days=('last7days_flg', 'sum')
    ).reset_index()

    # Group 'df_negative' by 'key' and aggregate counts
    rejected_grouped = df_negative.groupby(key).agg(
        negative_count=('Issue ID', 'count'),
        negative_count_last7days=('last7days_flg', 'sum')
    ).reset_index()

    # Merge passed and rejected counts on 'key'
    df = pd.merge(passed_grouped, rejected_grouped, on=key, how='outer').fillna(0)

    # Convert counts to integers
    df[['positive_count', 'positive_count_last7days', 'negative_count', 'negative_count_last7days']] = df[['positive_count', 'positive_count_last7days', 'negative_count', 'negative_count_last7days']].astype(int)

    # Compute 'total_count' and 'total_count_last7days'
    df['total_count'] = df['positive_count'] + df['negative_count']
    df['total_count_last7days'] = df['positive_count_last7days'] + df['negative_count_last7days']

    # Compute 'positive_ratio' (Overall Pass Rate)
    df['positive_ratio'] = (df['positive_count'] / df['total_count']) * 100
    df['positive_ratio'] = df['positive_ratio'].round(1)
    df['positive_ratio'] = df['positive_ratio'].fillna(0)  # Replace NaN with 0 for pass rate

    # Optional: Compute 'positive_ratio_last7days' if needed
    df['positive_ratio_last7days'] = (df['positive_count_last7days'] / df['total_count_last7days']) * 100
    df['positive_ratio_last7days'] = df['positive_ratio_last7days'].round(1)

    # Create formatted columns for 'total_count' and 'positive_count' with last 7 days counts
    df['total_count'] = df.apply(lambda row: format_count(row['total_count'], row['total_count_last7days']), axis=1)
    df['positive_count'] = df.apply(lambda row: format_count(row['positive_count'], row['positive_count_last7days']), axis=1)

    return df[[key, 'total_count', 'positive_count', 'positive_ratio']].rename(
        columns={'total_count':f'{prefix}_total_count',
                'positive_count':f'{prefix}_positive_count',
                'positive_ratio':f'{prefix}_positive_ratio'}
    )

def calculate_kpis(dfs, key):
    candidates_grouped = dfs['candidates'].groupby(key).agg(
        total_candidates=('Issue ID', 'count'),
        candidates_last7days=('last7days_flg', 'sum')
    ).reset_index()

    applied_grouped = dfs['applied'].groupby(key).agg(
        total_applied=('Issue ID', 'count'),
        applied_last7days=('last7days_flg', 'sum')
    )

    # Merge the two grouped DataFrames on key
    merged_df = pd.merge(candidates_grouped, applied_grouped, on=key, how='outer').fillna(0)

    # Ensure counts are integers
    merged_df[['total_candidates', 'candidates_last7days', 'total_applied', 'applied_last7days']] = merged_df[['total_candidates', 'candidates_last7days', 'total_applied', 'applied_last7days']].astype(int)

    # Create formatted columns for '候補者数' and '応募者数'
    merged_df['候補者数'] = merged_df.apply(lambda row: format_count(row['total_candidates'], row['candidates_last7days']), axis=1)
    merged_df['応募者数'] = merged_df.apply(lambda row: format_count(row['total_applied'], row['applied_last7days']), axis=1)

    # Select and reorder the desired columns
    final_df = merged_df[[key, '候補者数', '応募者数']]

    # List of stages to process
    stages = [
        ('screening_passed', 'screening_rejected', 'screening'),
        ('interview1_passed', 'interview1_rejected', 'interview1'),
        ('final_passed', 'final_rejected', 'final'),
        ('offer_accepted', 'offer_rejected', 'offer')
    ]

    # Iterate over each stage and merge the results
    for passed_key, rejected_key, prefix in stages:
        # Check if the DataFrames exist in 'dfs'
        if passed_key in dfs and rejected_key in dfs:
            stage_df = calculate_pass_rate(dfs[passed_key], dfs[rejected_key], prefix, key)
            final_df = pd.merge(final_df, stage_df, on=key, how='outer').fillna(0)
        else:
            # If DataFrames are missing, fill in zeros for the stage
            stage_df = pd.DataFrame({key: final_df[key]})
            stage_df[f'{prefix}_total_count'] = '0'
            stage_df[f'{prefix}_positive_count'] = '0'
            stage_df[f'{prefix}_positive_ratio'] = 0.0
            final_df = pd.merge(final_df, stage_df, on=key, how='outer').fillna(0)

    # Rename columns with appropriate names before return
    return final_df.rename(columns={
        'screening_total_count':'書類選考件数',
        'screening_positive_count':'書類選考通過数',
        'screening_positive_ratio':'書類選考通過率',
        'interview1_total_count':'1次面接実施件数',
        'interview1_positive_count':'1次面接通過数',
        'interview1_positive_ratio':'1次面接通過率',
        'final_total_count':'最終面接実施件数',
        'final_positive_count':'最終面接通過件数',
        'final_positive_ratio':'最終面接通過率',
        'offer_total_count':'内定通知件数',
        'offer_positive_count':'内定受諾件数',
        'offer_positive_ratio':'内定受諾率'
    })

In [4]:
# @title Execute
dataframes = {
    'candidates': get_issues_with_flag('candidates'),
    'applied': get_issues_with_flag('applied'),
    'screening_passed': get_issues_with_flag('screening_passed'),
    'screening_rejected': get_issues_with_flag('screening_rejected'),
    'interview1_passed': get_issues_with_flag('interview1_passed'),
    'interview1_rejected': get_issues_with_flag('interview1_rejected'),
    'final_passed': get_issues_with_flag('final_passed'),
    'final_rejected': get_issues_with_flag('final_rejected'),
    'offer_accepted': get_issues_with_flag('offer_accepted'),
    'offer_rejected': get_issues_with_flag('offer_rejected')
}

# Calculate KPIs for each stage by 採用チャネル
df_kpi = calculate_kpis(dataframes, '採用チャネル')

# Merge df_kpi with df_category on 採用チャネル
# df_kpi_merged = pd.merge(df_kpi, df_category, on='採用チャネル', how='left').drop(columns='カテゴリ',axis=1)
df_kpi_merged = pd.merge(df_kpi, df_category, on='採用チャネル', how='left')


## 以下の出力結果をコピーしてConfluence等にペーストしてください
- 数字の見方: base_dateより前の期間の件数 (+base_dateから7日間前までの件数)
> 例: 応募者数: 280 (+6) で base_date=2024-12-06 の場合<br>
> 集計開始〜12/6まで280人が応募しており、そのうち6名は11/30-12/6までに応募している

In [5]:
# @title 採用チャネル別 - エージェント
df_kpi_merged[df_kpi_merged['カテゴリ'] == 'エージェント'].drop(columns='カテゴリ',axis=1)

,採用チャネル,候補者数,応募者数,書類選考件数,書類選考通過数,書類選考通過率,1次面接実施件数,1次面接通過数,1次面接通過率,最終面接実施件数,最終面接通過件数,最終面接通過率,内定通知件数,内定受諾件数,内定受諾率
1,Asuka Professionals,4 (+2),5 (+3),4 (+2),4 (+2),100.0,2,2,100.0,1,1,100.0,1,0,0.0
4,KVCパートナーズ,4,4,3,1,33.3,1,0,0.0,0,0,0.0,0,0,0.0
9,bloom,5,5,5,4,80.0,3,1,33.3,1,0,0.0,0,0,0.0
10,doda,59 (+1),58 (+1),55,26,47.3,25,10,40.0,9,3,33.3,3,2,66.7
12,エグゼクティブリンク,2,2,3,2,66.7,0,0,0.0,0,0,0.0,0,0,0.0
14,フォルトナ,13 (+1),13 (+1),9 (+2),3,33.3,4,2,50.0,1,0,0.0,0,0,0.0
15,プリセプト,2,2,2,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0
17,リクナビ,286 (+6),287 (+7),261 (+3),75,28.7,69 (+5),24 (+1),34.8,21,12,57.1,7,6,85.7


In [6]:
# @title 採用チャネル別 - スカウト
df_kpi_merged[df_kpi_merged['カテゴリ'] == 'スカウト'].drop(columns='カテゴリ',axis=1)

,採用チャネル,候補者数,応募者数,書類選考件数,書類選考通過数,書類選考通過率,1次面接実施件数,1次面接通過数,1次面接通過率,最終面接実施件数,最終面接通過件数,最終面接通過率,内定通知件数,内定受諾件数,内定受諾率
0,AMBI,24 (+1),3,2,2,100.0,2,1,50.0,2,2,100.0,2,2,100.0
2,Facebook,2,2,2,2,100.0,2,1,50.0,1,1,100.0,1,1,100.0
3,Green,39 (+1),17,9,6,66.7,7,3,42.9,4 (+2),1 (+1),25.0,1 (+1),1 (+1),100.0
5,LinkedIn,11 (+2),8 (+2),7 (+2),6 (+2),85.7,3 (+1),2 (+1),66.7,2,1,50.0,1,1,100.0
6,OpenWork,36,9,8,5,62.5,4,2,50.0,2,1,50.0,0,0,0.0
7,Paiza,5,1,1,1,100.0,1,0,0.0,0,0,0.0,0,0,0.0
8,Wantedly,39 (+3),24 (+1),22 (+1),15 (+1),68.2,14,10,71.4,9,6,66.7,6,5,83.3
11,indeed,1,1,1,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0
13,ビズリーチ,95 (+5),51 (+5),34 (+1),18 (+1),52.9,19,11,57.9,13,6,46.2,3,1,33.3


In [7]:
# @title 応募ポジション別
calculate_kpis(dataframes, 'ポジション')

,ポジション,候補者数,応募者数,書類選考件数,書類選考通過数,書類選考通過率,1次面接実施件数,1次面接通過数,1次面接通過率,最終面接実施件数,最終面接通過件数,最終面接通過率,内定通知件数,内定受諾件数,内定受諾率
0,AIコンサルタント,7,3,3,2,66.7,2,1,50.0,1,0,0.0,0,0,0.0
1,ITコンサルタント,177 (+11),154 (+10),136 (+5),59 (+4),43.4,56 (+3),28 (+1),50.0,25,16,64.0,14,11,78.6
2,ITコンサルタント(PM),101 (+3),96 (+4),82 (+4),23,28.0,24 (+1),9,37.5,11,6,54.5,3,2,66.7
3,ITコンサルタント（セキュリティ）,3,2,2,1,50.0,0,0,0.0,0,0,0.0,0,0,0.0
4,その他,1,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0
5,アプリケーションエンジニア,18,17,16,4,25.0,3,1,33.3,1,1,100.0,1,1,100.0
6,シニアアプリケーションエンジニア,10,10,10,2,20.0,2,1,50.0,1,1,100.0,1,1,100.0
7,シニアデータサイエンティスト,61 (+1),47 (+1),32,20,62.5,18 (+1),8,44.4,8 (+1),3,37.5,2,1,50.0
8,データサイエンティスト,217 (+7),134 (+5),115 (+2),53 (+2),46.1,45 (+1),20 (+1),44.4,19 (+1),7 (+1),36.8,4 (+1),3 (+1),75.0
9,データ戦略コンサルタント,9,9,9,1,11.1,1,0,0.0,0,0,0.0,0,0,0.0
